In [156]:
import librosa
import os
import numpy as np
import pandas as pd
import thinkdsp
import codecs
import json

from importlib import reload
reload(thinkdsp)

def get_audio_sample_rate_text(path_to_audio, path_to_text):
    try:
        X, sample_rate = librosa.load(path_to_audio,)
    except FileNotFoundError:
        return None
    with open(path_to_text, 'r', encoding="utf-8") as reader:
        text = reader.read()
    return X, sample_rate, text

def read_json(json_path):
    content = []
    for json_line in codecs.open(json_path, 'r', 'utf-8'):
        try:
            json_data = json.loads(json_line)
        except Exception as exc:
            print(json_path)
            raise IOError("Error reading json: %s" % str(exc))
        content.append(json_data)
    return content


def read_manifest(manifest_path, max_duration=float('inf'), min_duration=0.0):
    manifest = []
    json_data = read_json(manifest_path)
    for json_line in json_data:
        if min_duration <= json_line["duration"] <= max_duration:
            manifest.append(json_line)
    return manifest

PATH_TO_DATA = "../../ru_Open_STT_manifest_preparation/converted/"
MANIFEST = "../../ru_Open_STT_manifest_preparation/converted/train_private_buriy_books.json"

In [148]:
pd.read_csv("news_1200.csv", sep=";", header=None)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,NaN,Unnamed: 0,article_id,category,title,tags,text,date,char_len,token_len,lowered_text,filtered_text,tokens,filtered_tokens,normalized_tokens,processed_text,proc_token_len,vocab_tokens,vocab_text
1,2357.0,2357,https://ria.ru/20201222/neprikosnovennost-1590...,POLITICS,Путин подписал закон о гарантиях неприкосновен...,"['Андрей Клишас', 'Совет Федерации РФ', 'Госду...","МОСКВА, 22 дек — РИА Новости. Владимир Путин п...",2020-12-22,1481,189,"москва, 22 дек — риа новости. владимир путин п...",москва дек риа новости владимир путин подпис...,"['москва', 'дек', 'риа', 'новости', 'владимир'...","['москва', 'дек', 'риа', 'новости', 'владимир'...","['москва', 'дек', 'риа', 'новость', 'владимир'...",москва дек риа новость владимир путин подписат...,138,"['подписать', 'закон', 'семья', 'соответствующ...",подписать закон семья соответствующий документ...
2,1878.0,1878,https://ria.ru/20201007/kamchatka-1578551098.html,SCIENCE,Ученые из ДВФУ исследуют ситуацию с загрязнени...,"['Наука', 'Загрязнение воды на Камчатке', 'Сле...","ВЛАДИВОСТОК, 7 окт – РИА Новости. Ученые Дальн...",2020-07-10,3247,426,"владивосток, 7 окт – риа новости. ученые дальн...",владивосток окт риа новости ученые дальневос...,"['владивосток', 'окт', 'риа', 'новости', 'учен...","['владивосток', 'окт', 'риа', 'новости', 'учен...","['владивосток', 'окт', 'риа', 'новость', 'учён...",владивосток окт риа новость учёный дальневосто...,314,"['учёный', 'федеральный', 'университет', 'пров...",учёный федеральный университет проводить собст...
3,2698.0,2698,https://ria.ru/20201125/putin-1586269457.html,POLITICS,Путин из-за нелетной погоды отложил поездку в ...,"['Роснефть', 'Дмитрий Песков', 'Владимир Путин...","МОСКВА, 25 ноя - РИА Новости. Президент России...",2020-11-25,570,77,"москва, 25 ноя - риа новости. президент россии...",москва ноя риа новости президент россии влад...,"['москва', 'ноя', 'риа', 'новости', 'президент...","['москва', 'ноя', 'риа', 'новости', 'президент...","['москва', 'ной', 'риа', 'новость', 'президент...",москва ной риа новость президент россия владим...,60,"['рабочий', 'изз', 'журналист', 'пресссекретар...",рабочий изз журналист пресссекретарить государ...
4,1732.0,1732,https://ria.ru/20201020/nokia-1580632725.html,SCIENCE,Nokia предлагает новые кнопочные телефоны по ц...,"['Наука', 'Nokia Corporation', 'гаджеты', 'Тех...","МОСКВА, 20 окт — РИА Новости. Компания HMD Glo...",2020-10-20,814,123,"москва, 20 окт — риа новости. компания hmd glo...",москва окт риа новости компания объявила о...,"['москва', 'окт', 'риа', 'новости', 'компания'...","['москва', 'окт', 'риа', 'новости', 'компания'...","['москва', 'окт', 'риа', 'новость', 'компания'...",москва окт риа новость компания объявить выход...,69,"['компания', 'объявить', 'выход', 'рынок', 'мо...",компания объявить выход рынок мобильный способ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,2254.0,2254,https://radiosputnik.ria.ru/20201229/peskov-15...,POLITICS,"""Живой интерес"". Песков счел ""рассуждениями"" с...","['Алексей Дюмин', 'Алексей Кудрин', 'Совет Фед...","МОСКВА, 29 декабря/ Радио Sputnik. Пресс-секре...",2020-12-29,1138,153,"москва, 29 декабря/ радио sputnik. пресс-секре...",москва декабря радио пресссекретарь президен...,"['москва', 'декабря', 'радио', 'пресссекретарь...","['москва', 'декабря', 'радио', 'пресссекретарь...","['москва', 'декабрь', 'радио', 'пресссекретари...",москва декабрь радио пресссекретарить президен...,109,"['декабрь', 'радио', 'пресссекретарить', 'дмит...",декабрь радио пресссекретарить дмитрий песок п...
1197,1402.0,1402,https://ria.ru/20201126/kovid-1586432867.html,SCIENCE,Ученые нашли гены предрасположенности к тяжело...,"['Наука', 'Коронавирус COVID-19', 'генетика', ...","МОСКВА, 26 ноя — РИА Новости. Американские уче...",2020-11-26,3096,410,"москва, 26 ноя — риа новости. американские уче...",москва ноя риа новости американские ученые п...,"['москва', 'ноя', 'риа', 'новости', 'аме

In [123]:
#test_manifest = read_manifest(MANIFEST)
#test_augment_manifest = test_manifest[:int(len(test_manifest)/5)]

In [157]:
train_manifest = read_manifest(MANIFEST)
train_augment_manifest = train_manifest[int(len(train_manifest)/5) : int(len(train_manifest)/5) + int(len(train_manifest)/2)]
len(train_augment_manifest)

25584

In [160]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, Trim, AddShortNoises, Normalize

def create_augmented_data(path_to_data, manifest, path_output, sr=16000): #path_output = "train_augmented_buriy_books_20%/"
    output_man = []
    for m in manifest:
        wav_name = m["audio_filepath"]
        augmented_wav_name = wav_name.split('/')[1].split('.wav')[0]
        audio, sr = librosa.load(path_to_data + wav_name, sr)
        
        augmentations = {
                         #"Trim" : Trim(top_db=20, p=1),
                         #"Normalize" : Normalize(p=1),
                         "AddGaussianNoise" : AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.06, p=1),
                         "Shift" : Shift(min_fraction=-0.5, max_fraction=0.5, p=1),
                         "AddShortNoises" : AddShortNoises(sounds_path="../../labs/audio-noise-analysis/data/noise/", 
                                                           p=1.0, burst_probability=1, lru_cache_size=1024)}
        
        for name, augmentor in augmentations.items():
            augment_method = Compose([augmentor])
            augmented_samples = augment_method(samples=audio, sample_rate=sr)
            augmented_wave = thinkdsp.Wave(augmented_samples, framerate=sr)
            augmented_wave.write(path_output + augmented_wav_name + "_" + name.lower() + "_.wav")
            output_man.append({"audio_filepath" : path_output + augmented_wav_name + "_" + name.lower() + "_.wav", "duration" : m["duration"], "text" : m["text"]})

    with codecs.open(path_output + "train_augmented_private_buriy_books_20%_70%.json", 'w', encoding='utf-8') as fout:
        for m in output_man:
            fout.write(json.dumps(m, ensure_ascii=False) + '\n')

In [170]:
from audiomentations import AddShortNoises, AddGaussianNoise, Shift, Compose
import thinkdsp
from tqdm import tqdm
def create_augmented_data_common_voice(manifest, path_output, output_manifest_name, sr=16000): #path_output = "train_augmented_buriy_books_20%/"
    output_man = []
    for m in tqdm(manifest):
        wav_name = m["audio_filepath"]        
        
        augmented_wav_name = wav_name.replace("dev", "dev_augmented").split('.wav')[0]
        audio, sr = librosa.load(wav_name, sr)
        
        augmentations = {
                         #"Trim" : Trim(top_db=20, p=1),
                         #"Normalize" : Normalize(p=1),
                         "AddGaussianNoise" : AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.03, p=1),
                         "Shift" : Shift(min_fraction=-0.5, max_fraction=0.5, p=1),
                         "AddShortNoises" : AddShortNoises(sounds_path="../../labs/audio-noise-analysis/data/noise/", 
                                                           p=1.0, burst_probability=1, lru_cache_size=1024)}
        
        for name, augmentor in augmentations.items():
            augment_method = Compose([augmentor])
            augmented_samples = augment_method(samples=audio, sample_rate=sr)
            augmented_wave = thinkdsp.Wave(augmented_samples, framerate=sr)
            augmented_wave.write(augmented_wav_name + "_" + name.lower() + "_.wav")
            output_man.append({"audio_filepath" : augmented_wav_name + "_" + name.lower() + "_.wav", "duration" : m["duration"], "text" : m["text"]})
    with codecs.open(path_output + output_manifest_name, 'w', encoding='utf-8') as fout:
        for m in output_man:
            fout.write(json.dumps(m, ensure_ascii=False) + '\n')

In [180]:
test_manifest = read_manifest("../../ru_Open_STT_manifest_preparation/common_voice/test.json")

In [182]:
test_manifest_augmented = read_manifest("../../ru_Open_STT_manifest_preparation/common_voice/test_augmented.json")

In [183]:
from random import shuffle
shuffle(test_manifest_augmented)

In [185]:
with codecs.open("../../ru_Open_STT_manifest_preparation/common_voice/test_augmented_1000_before.json", 'w', encoding='utf-8') as fout:
    for m in test_manifest_augmented[:1000]:
        fout.write(json.dumps(m, ensure_ascii=False) + '\n')

#with codecs.open("../../ru_Open_STT_manifest_preparation/common_voice/dev_augmented_8000_after.json", 'w', encoding='utf-8') as fout:
#    for m in test_manifest_augmented[8000:]:
#        fout.write(json.dumps(m, ensure_ascii=False) + '\n')

In [181]:
create_augmented_data_common_voice(test_manifest, "../../ru_Open_STT_manifest_preparation/common_voice/test_augmented/", "../test_augmented.json")

100%|██████████████████████████████████████████████████████████████████████████████| 8007/8007 [13:13<00:00, 10.10it/s]


In [161]:
create_augmented_data(PATH_TO_DATA, train_augment_manifest, PATH_TO_DATA + "train_augmented_buriy_books_20%_70%/")

In [163]:
audio, sr = librosa.load(PATH_TO_DATA + test_manifest[0]["audio_filepath"], sr)
augment = Compose([
    #Trim(top_db=20, p=0.25),
    #Normalize(p=0.25),
    #AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.06, p=0.25),
    Shift(min_fraction=-0.5, max_fraction=0.5, p=1),
    #AddShortNoises(sounds_path="../../labs/audio-noise-analysis/data/noise/", p=1.0, burst_probability=1, lru_cache_size=1024),

    #TimeStretch(min_rate=0.8, max_rate=1.25, p=1.0),
    #PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
])

augmented_samples = augment(samples=audio, sample_rate=sr)
augmented_wave = thinkdsp.Wave(augmented_samples, framerate=sr)
augmented_wave.make_audio()

In [113]:
w = augmented_wave.write("../../labs/audio-noise-analysis/augmented_wave_.wav")